In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using OrdinaryDiffEq
using ForwardDiff
using Plots

In [ ]:
#Pendulum Parameters
ℓ = 1.0
m = 1.0
g = 9.81
c = 0.25

In [ ]:
#Reference Solution w/ODE Solver

#Classical pendulum dynamics
function f(x,p,t)
    q = x[1]
    v = x[2]
    
    v̇ = -(g/ℓ)*sin(q) - c*v
    
    ẋ = [v; v̇]
end

#initial conditions
x0 = [pi/2; 0]

#Simulate
tspan = (0.0,50.0)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5());
#sol = solve(prob,Tsit5(),abstol=1e-5,reltol=1e-5);
#sol = solve(prob,Tsit5(),abstol=1e-6,reltol=1e-6);

In [ ]:
plot(sol,idxs=(0,1))

In [ ]:
#Energy Functions

function T(θ̇)
    0.5*m*ℓ*ℓ*θ̇*θ̇
end

function U(θ)
    m*g*ℓ*(1-cos(θ))
end

function H(x)
    U(x[1]) + T(x[2])
end

In [ ]:
#Define discrete Lagrangian and DEL equation

function L(q,q̇)
    T(q̇) - U(q)
end

function Ld(q1,q2)
    θm = 0.5*(q1+q2)
    θ̇m = (q2-q1)/h
    return h*L(θm,θ̇m)
end

function DEL(q1,q2,q3)
    ForwardDiff.derivative(dq2->Ld(q1,dq2),q2) + ForwardDiff.derivative(dq2->Ld(dq2,q3),q2) - (h/2)*c*(q2-q1)/h - (h/2)*c*(q3-q2)/h
end

In [ ]:
#Simulate using DEL equation

h = 0.05 #time step
tk = 0:h:tspan[2]

#Initial conditions taken from reference solution
q1 = sol(tk[1])[1]
q2 = sol(tk[2])[1]

qhist = zeros(length(tk))
qhist[1] = q1
qhist[2] = q2

for k = 2:(length(tk)-1)
    qhist[k+1] = qhist[k]
    r = DEL(qhist[k-1],qhist[k],qhist[k+1])
    
    #Newton's method
    while norm(r) > 1e-12
        R = ForwardDiff.derivative(dq3->DEL(qhist[k-1],qhist[k],dq3),qhist[k+1])
        qhist[k+1] = qhist[k+1] - R\r
        r = DEL(qhist[k-1],qhist[k],qhist[k+1])
    end
end

In [ ]:
plot(tk,qhist)
plot!(sol,idxs=(0,1))